In [1]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import pickle

# w2v_embeddings = KeyedVectors.load("word2vec_first_embeddings")
w2v_model = Word2Vec.load("en_1000_no_stem/en.model")

In [3]:
w2v_model.wv

In [4]:
y_train, x_b, x_train = pickle.load(open("dbpedia_csv/" + "/train_preprocessed.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, x_tb, x_test = pickle.load(open("dbpedia_csv/" + "/test_preprocessed.pickle", "rb"))

In [5]:

split_examples =[x.split() for x in x_train]

In [6]:
split_test = [x.split() for x in x_test]

In [7]:
from time import time

In [8]:
import pandas
import numpy

In [9]:


def word_vector(tokens, size):
    vec = numpy.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += w2v_model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec


In [11]:

wordvec_arrays = numpy.zeros((len(split_examples), 1000)) 
for i in range(len(split_examples)):
    wordvec_arrays[i,:] = word_vector(split_examples[i], 1000)

In [12]:
wordvec_df = pandas.DataFrame(wordvec_arrays)
wordvec_df.shape

(560000, 1000)

In [13]:
x_train = wordvec_df
# x_test = wordvec_df[560000:]

In [14]:
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [15]:
wordvec_arrays = numpy.zeros((len(split_test), 1000)) 
for i in range(len(split_test)):
    wordvec_arrays[i,:] = word_vector(split_test[i], 1000)
    
wordvec_df = pandas.DataFrame(wordvec_arrays)

x_test = wordvec_df
x_test.shape

(70000, 1000)

array([[0.35181972, 0.32315985, 0.46015313, ..., 0.66905815, 0.20013444,
        0.65691885],
       [0.44253483, 0.37957672, 0.55933342, ..., 0.70472212, 0.40026887,
        0.64514688],
       [0.44253483, 0.37957672, 0.55933342, ..., 0.70472212, 0.40026887,
        0.64514688],
       ...,
       [0.44253483, 0.37957672, 0.55933342, ..., 0.70472212, 0.40026887,
        0.64514688],
       [0.43695842, 0.35707579, 0.59953573, ..., 0.88842044, 0.39346209,
        0.81294401],
       [0.38773754, 0.53181052, 0.64807791, ..., 0.50445289, 0.50654756,
        0.65116546]])

In [40]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3).fit(x_train, y_train)

accuracy = knn.score(x_test, y_test)
print(accuracy)

knn_predictions = knn.predict(x_test)
cm = confusion_matrix(y_test, knn_predictions)

cm

KeyboardInterrupt: 

In [44]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Choose either standardization or Normalization
# On this data Min Max Normalization is used because we need to fit Naive Bayes

# Choose between standardization and MinMAx normalization
#PredictorScaler=StandardScaler()
PredictorScaler=MinMaxScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(x_train+x_test)


x_train = PredictorScalerFit.transform(x_train)
x_test = PredictorScalerFit.transform(x_test)

# Generating the standardized values of X
# X=PredictorScalerFit.transform(X)

# # Split the data into training and testing set
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=428)

# # Sanity check for the sampled data
# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)

In [50]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB, MultinomialNB

# GaussianNB is used in Binomial Classification
# MultinomialNB is used in multi-class classification
#clf = GaussianNB()
clf = MultinomialNB()

# Printing all the parameters of Naive Bayes
print(clf)

NB=clf.fit(x_train,y_train)
prediction=NB.predict(x_test)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test, prediction, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

# Importing cross validation function from sklearn
from sklearn.model_selection import cross_val_score

# Running 10-Fold Cross validation on a given algorithm
# Passing full data X and y because the K-fold will split the data and automatically choose train/test
Accuracy_Values=cross_val_score(NB, x_train+x_test, y_train_y_test, cv=5, scoring='f1_weighted')
print('\nAccuracy values for 5-fold Cross Validation:\n',Accuracy_Values)
print('\nFinal Average Accuracy of the model:', round(Accuracy_Values.mean(),2))

MultinomialNB()
              precision    recall  f1-score   support

           1       0.00      0.00      0.00      5000
           2       0.00      0.00      0.00      5000
           3       0.00      0.00      0.00      5000
           4       0.00      0.00      0.00      5000
           5       0.16      0.08      0.11      5000
           6       0.50      0.01      0.02      5000
           7       0.20      0.00      0.00      5000
           8       0.00      0.00      0.00      5000
           9       0.20      0.24      0.22      5000
          10       0.08      0.95      0.15      5000
          11       0.00      0.00      0.00      5000
          12       0.15      0.03      0.05      5000
          13       0.20      0.05      0.08      5000
          14       0.00      0.00      0.00      5000

    accuracy                           0.10     70000
   macro avg       0.11      0.10      0.05     70000
weighted avg       0.11      0.10      0.05     70000

[[   0   

C:\Users\Ana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ana\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: operands could not be broadcast together with shapes (560000,50) (70000,50) 

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
t = time()

# naive_bayes_classifier = MultinomialNB()
# naive_bayes_classifier.fit(x_train, y_train)

classifier = RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 42, max_depth=20)
classifier.fit(x_train, y_train)

training_time = time() - t
print("train time: %0.3fs" % training_time)

train time: 3661.649s


In [18]:
t = time()
y_pred = classifier.predict(x_test)

test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred, target_names=[str(x) for x in range(14)]))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

test time:  2.633s
accuracy:   0.949
              precision    recall  f1-score   support

           0       0.90      0.87      0.89      5000
           1       0.95      0.96      0.96      5000
           2       0.89      0.84      0.86      5000
           3       0.98      0.98      0.98      5000
           4       0.94      0.96      0.95      5000
           5       0.96      0.97      0.96      5000
           6       0.95      0.94      0.94      5000
           7       0.97      0.98      0.98      5000
           8       1.00      0.98      0.99      5000
           9       0.98      0.98      0.98      5000
          10       0.99      0.98      0.98      5000
          11       0.95      0.97      0.96      5000
          12       0.96      0.96      0.96      5000
          13       0.87      0.91      0.89      5000

    accuracy                           0.95     70000
   macro avg       0.95      0.95      0.95     70000
weighted avg       0.95      0.95      0.95

## KNN and RandomForest with tf-idf


In [51]:
y_train, x_b, x_train = pickle.load(open("dbpedia_csv/" + "/train_preprocessed.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, x_tb, x_test = pickle.load(open("dbpedia_csv/" + "/test_preprocessed.pickle", "rb"))

In [52]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [53]:
t = time()  # not compulsory

# loading CountVectorizer
tf_vectorizer = CountVectorizer() # or term frequency

X_train_tf = tf_vectorizer.fit_transform(x_train)

duration = time() - t
print("Time taken to extract features from training data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

Time taken to extract features from training data : 14.024000 seconds
n_samples: 560000, n_features: 698699


In [54]:
t = time()
X_test_tf = tf_vectorizer.transform(x_test)

duration = time() - t
print("Time taken to extract features from test data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

Time taken to extract features from test data : 1.682999 seconds
n_samples: 70000, n_features: 698699


In [59]:
t = time()

# naive_bayes_classifier = MultinomialNB()
# naive_bayes_classifier.fit(x_train, y_train)

classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42, max_depth=20)
classifier.fit(X_train_tf, y_train)

training_time = time() - t
print("train time: %0.3fs" % training_time)

train time: 67.408s


In [60]:
t = time()
y_pred = classifier.predict(X_test_tf)

test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred, target_names=[str(x) for x in range(14)]))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

test time:  1.639s
accuracy:   0.895
              precision    recall  f1-score   support

           0       0.89      0.81      0.85      5000
           1       0.87      0.94      0.90      5000
           2       0.89      0.65      0.75      5000
           3       0.93      0.96      0.94      5000
           4       0.93      0.92      0.92      5000
           5       0.90      0.93      0.92      5000
           6       0.92      0.82      0.87      5000
           7       0.88      0.96      0.92      5000
           8       0.95      0.98      0.96      5000
           9       0.90      0.85      0.87      5000
          10       0.86      0.92      0.89      5000
          11       0.84      0.98      0.91      5000
          12       0.91      0.96      0.93      5000
          13       0.86      0.86      0.86      5000

    accuracy                           0.90     70000
   macro avg       0.90      0.90      0.89     70000
weighted avg       0.90      0.90      0.89

In [58]:
knn = KNeighborsClassifier(n_neighbors = 3, n_jobs=-1).fit(X_train_tf, y_train)

accuracy = knn.score(X_test_tf, y_test)
print(accuracy)

knn_predictions = knn.predict(X_test_tf)
cm = confusion_matrix(y_test, knn_predictions)

cm

KeyboardInterrupt: 